In [3]:
import sys
sys.path.insert(0, '../../cbc.rans/cbc')

In [4]:
from pdesys import *

SyntaxError: invalid syntax (PDESubSystems.py, line 160)

In [ ]:
# # Test for PETSc or Epetra
# if not has_la_backend("PETSc") and not has_la_backend("Epetra"):
#     info_red("DOLFIN has not been configured with Trilinos or PETSc. Exiting.")
#     exit()

# info_green("This demo is unlikely to converge if PETSc is not configured with Hypre or ML.")

# Boundaries
def right(x, on_boundary): return x[0] > (1.0 - DOLFIN_EPS)
def left(x, on_boundary): return x[0] < DOLFIN_EPS
def top_bottom(x, on_boundary):
    return x[1] > 1.0 - DOLFIN_EPS or x[1] < DOLFIN_EPS
        
class Stokes_Solver(PDESystem):
    
    def __init__(self, mesh, parameters=parameters):
        PDESystem.__init__(self, [['u', 'p']], mesh, parameters)
        self.setup() # Create functionspaces, trial/testfunctions and solution functions
        
        self.f = Constant((0.0, 0.0, 0.0))
        
        # Set up boundary conditions.
        inflow = Expression(("-sin(x[1]*pi)", "0.0", "0.0"))
        noslip = Constant((0.0, 0.0, 0.0))
        zero = Constant(0)
        self.bc['up'] = [DirichletBC(self.V['up'].sub(0), noslip, top_bottom),
                         DirichletBC(self.V['up'].sub(0), inflow, right),
                         DirichletBC(self.V['up'].sub(1), zero, left)] 
        
        # Set up variational problem
        self.define()
        
    def define(self):
        name = 'Stokes_' + str(self.prm['pdesubsystem']['up'])
        self.pdesubsystems['up'] = eval(name)(vars(self), ['u', 'p'], 
                                               bcs=self.bc['up'])
                        
class Stokes_1(PDESubSystem):
    """Variational form of the Stokes problem."""
    def form(self, u, u_, v_u, p, p_, v_p, f, **kwargs):
        self.prm['assemble_system'] = True # Assemble symmetric system with assemble_system
        return inner(grad(u), grad(v_u))*dx + div(v_u)*p*dx + v_p*div(u)*dx \
               - inner(f, v_u)*dx
                       
    def get_precond(self, u, v_u, p, v_p, f, **kwargs):
        # Form for use in constructing preconditioner matrix
        b = inner(grad(u), grad(v_u))*dx + p*v_p*dx
        L = inner(f, v_u)*dx
        
        # Assemble preconditioner system. B is hooked up in setup_solver
        B, dummy = assemble_system(b, L, self.bcs)
        
        return B